In [70]:
import numpy as np
import random
import regex as re 
import os
import pandas as pd

In [66]:
def split_txt(path, low=200, high=500, ignore=20, strip=True):
    # path指向的是txt文件，返回分割后的list，每个item为长度在[low, high]的文本
    res, curr_txt, curr_limit = [], '', random.randint(low, high)
    alpha = re.compile(r'[a-z]')
    with open(path, 'r') as f:
        for line in f.readlines()[ignore:]:
            line = line.strip() if strip else line
            if len(alpha.findall(line)) > 0:
                continue
            curr_txt += line
            if len(curr_txt) > curr_limit:
                res.append(curr_txt)
                curr_txt, curr_limit = '', random.randint(low, high)
    
    return res

def split_dir(dir='../data/original/', class_num=1):
    # 获取某一类所有txt分割文件

    dir_path = os.path.join(dir, str(class_num))
    x, labels = [], []
    for f in os.listdir(dir_path):
        if f[-3:] != 'txt':
            continue
        path = os.path.join(dir_path, f)
        x += split_txt(path)
    labels = [(class_num-1)] * len(x)

    return x, labels

def to_csv(train_radio=0.8, test_radio=0.1, shuffle=False):
    assert(train_radio + test_radio < 1)
    train_x, test_x, valid_x = [], [], []
    train_y, test_y, valid_y = [], [], []
    for i in range(3):
        x, y = split_dir(class_num=i+1)
        if shuffle:
            random.shuffle(x)
        parti_1, parti_2 = int(len(x)*train_radio), int(len(x) * (train_radio+test_radio))
        train_x, test_x, valid_x = train_x + x[:parti_1], test_x + x[parti_1:parti_2], valid_x + x[parti_2:]
        train_y, test_y, valid_y = train_y + y[:parti_1], test_y + y[parti_1:parti_2], valid_y + y[parti_2:]
    
    train_df = pd.DataFrame(data={'txt':train_x, 'label':train_y})
    test_df = pd.DataFrame(data={'txt':test_x, 'label':test_y})
    valid_df = pd.DataFrame(data={'txt':valid_x, 'label':valid_y})

    train_df.to_csv('../data/csv/train.csv', quoting=1, line_terminator="\r\n")
    test_df.to_csv('../data/csv/test.csv', quoting=1, line_terminator="\r\n")
    valid_df.to_csv('../data/csv/valid.csv', quoting=1, line_terminator="\r\n")

In [67]:
to_csv(0.8, 0.1)

In [68]:
test = pd.read_csv('../data/csv/test.csv')

In [69]:
test.head(5)

,Unnamed: 0,txt,label
0,0,旁观众高手固觉惊异，任我行心下更是骇然。十余年前任我行左冷禅剧斗，未曾使用“ 吸星大法”，已...,0
1,1,这一场拚斗，面子上似是未分胜败，但任左二人内心均知，自己的武功之中具有极大弱点，当日不输，实...,0
2,2,便在心念电闪之际，噗的一声响，左冷禅的手指已戳中他左胸“天池穴”。旁观众人啊的一声，齐声呼叫...,0
3,3,第二十七章 三战（5）向问天冷笑道：“岳先生能言善辩，令人好生佩服，只不过和‘君子’二字，未...,0
4,4,任我行道：“冲虚道长在贵方是生力军，我们这一边也得出一个生力军才是。”抬头叫道：“令狐冲小兄...,0
